In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy

# linear

In [2]:
M = 0.5
m = 0.2
F0 = 0.1
F1 = 0.1
I = 0.006
g = 9.8
l = 0.3
dt = 0.01

# M,m,F0,F1,I,g,l,dt = sympy.symbols("M m F0 F1 J g l dt")
e1,e2,e3,e4 = sympy.symbols("e1 e2 e3 e4")


p = I*(M+m)+M*m*l**2 # denominator for the A and B matrices
A = (M+m)*m*g*l/p
B = -F1*(M+m)/p
C = F0*m*l/p
D = -m*l/p
E = -m*m*g*l*l/p
F = F1*m*l/p
G = -F0*(I+m*l*l)/p
H = (I+m*l*l)/p

# p,A,B,C,D,E,F,G,H,dt = sympy.symbols("p A B C D E F G H dt")


kp1 = 240.00
kd1 = 34.10
kp2 = 60.00
kd2 = 79.18

# kp1,kd1,kp2,kd2 = sympy.symbols("kp1 kd1 kp2 kd2")
x_d_r = ( kp1*e1+kd1*e2+A*e1+B*e2-E*D/H*e1-F*D/H*e2-G*D/H*e4-D/H*kd2*e4-D/H*kp2*e3) / C
# u = E/H*e1 +F/H*e2+G/H*e4 +(kp2*dt)*x_d_r/H +kd2/H*e4
u = E/H*e1 +F/H*e2+G/H*e4 +kp2*e3/H +kd2/H*e4

e2_d = A*e1+B*e2+C*e4-D*u
e4_d = -kp2*e3 -kd2*e4
x_d_r_d = ( kp1*e2+kd1*e2_d+A*e2+B*e2_d-E*D/H*e2-F*D/H*e2_d-G*D/H*e4_d-D/H*kd2*e4_d-D/H*kp2*e4) / (C + D/H*kp2*dt)
e3_d = x_d_r_d * dt + e4


j21 = sympy.diff(e2_d,e1)
j22 = sympy.diff(e2_d,e2)
j23 = sympy.diff(e2_d,e3)
j24 = sympy.diff(e2_d,e4)

j31 = sympy.diff(e3_d,e1)
j32 = sympy.diff(e3_d,e2)
j33 = sympy.diff(e3_d,e3)
j34 = sympy.diff(e3_d,e4)

JM = sympy.Matrix([[0, 1, 0, 0],
              [j21,j22,j23,j24],
              [j31,j32,j33,j34],
              [0, 0, -kp2, -kd2]])
JM

Matrix([
[               0,                 1,                0,               0],
[            24.5, -4.16666666666667,            150.0,          197.95],
[-7.0148115942029, -1.33700483091787, 70.3973913043479, 92.466308115942],
[               0,                 0,            -60.0,          -79.18]])

In [ ]:
JM.eigenvects()

# nonlinear 

In [6]:
import sympy
e1, e2, e3, e4 = sympy.symbols("e1 e2 e3 e4")
# M,m,F0,F1,J,g,l,dt = sympy.symbols("M m F0 F1 J g l dt")

M = .5
m = 0.2
F0 = 0.1
F1 = 0.1
J = 0.006
g = 9.8
l = 0.3
dt = 0.01

kp1 = 240.00
kd1 = 34.10
kp2 = 60.00
kd2 = 79.18

p = (M+m)*(J+m*l**2) - m**2*l**2*sympy.cos(e1)**2
p_d = 2*m*m*l*l*sympy.cos(e1)*sympy.sin(e1)*e2

A = p*(kp1*e1+kd1*e2) - F1*(M+m)*e2 + (M+m)*m*g*l*sympy.sin(e1)
A += m*l*sympy.cos(e1)/(J+m*l*l) * (F1*m*l*e2*sympy.cos(e1) -F0*(J+m*l*l)*e4 -m*m*g*l*l*sympy.sin(e1)*sympy.cos(e1))
A += m*l*sympy.cos(e1)*(p/(J+m*l*l)*(kd2*e4+kp2*e3))
B = F0*m*l*sympy.cos(e1) # -p*m*l*sympy.cos(e1)*kp2*dt/(J+m*l*l)

u = 1/(J+m*l*l) * (F1*m*l*e2*sympy.cos(e1) +(J+m*l*l)*m*l*e2*e2*sympy.sin(e1)\
                   -F0*(J+m*l*l)*e4 \
                   -m*m*g*l*l*sympy.sin(e1)*sympy.cos(e1)\
                   +p*kp2*e3 \
                   +p*kd2*e4 )

e1_d = e2
e2_d = -F1/p*(M+m)*e2 -m*m*l*l/p*e2*e2*sympy.sin(e1)*sympy.cos(e1)+F0*m*l/p*sympy.cos(e1)*e4 +(M+m)*m*g*l/p*sympy.sin(e1) +m*l/p*sympy.cos(e1)*u
e4_d = -kp2*e3-kd2*e4

A_d = p_d*kp1*e1+p*kp1*e2 +p_d*kd1*e2+p*kd1*e2_d -F1*(M+m)*e2_d+(M+m)*m*g*l*sympy.cos(e1)*e2
A_d += m*l/(J+m*l*l)*( F1*m*l*(e2_d*sympy.cos(e1)*sympy.cos(e1)-e2*2*sympy.cos(e1)*sympy.sin(e1)*e2)\
                      - F0*(J+m*l*l)*(e4_d*sympy.cos(e1)-e4*sympy.sin(e1)*e2)\
                      -m*m*g*l*l*( -sympy.sin(e1)*e2*sympy.sin(2*e1)/2 +sympy.cos(e1)*sympy.cos(2*e1)*e2))

A_d += m*l*kd2/(J+m*l*l)*((p_d*e4+p*e4_d)*sympy.cos(e1)+p*e4*(-sympy.sin(e1))*e2)
A_d += m*l*kp2/(J+m*l*l)*((p_d*e3+p*e4)*sympy.cos(e1)+p*e3*(-sympy.sin(e1))*e2)

B_d = -F0*m*l*sympy.sin(e1)*e2 # - m*l*kp2*dt/(J+m*l*l)*(p_d*sympy.cos(e1)-p*sympy.sin(e1)*e2)

p3 = p*m*l*sympy.cos(e1)*kp2*dt/(J+m*l*l)
x3_d = (A_d*B-B_d*A)/(B*B-p3*B) * dt + e4

#x_d_r = p*(kp1*e1+kd1*e2) - F1*(M+m)*e2 + m*m*l*l*e2*sympy.sin(e1)*sympy.cos(e1) - (M+m)*m*g*l*sympy.sin(e1) + F0*m*l*sympy.cos(e1)*e4
#x_d_r += m*l*sympy.cos(e1)/(J+m*l*l) * (F1*m*l*e2*sympy.cos(e1) +(J+m*l*l)*m*l*e2**2*sympy.sin(e1) -F0*(J+m*l*l)*e4 -m*m*g*l*l*sympy.sin(e1)*sympy.cos(e1))
#x_d_r += m*l*sympy.cos(e1)*(p/(J+m*l*l)*kd2*e4)
#x_d_r /= (F0*m*l*sympy.cos(e1)-p*m*l*sympy.cos(e1)*kp2*dt/(J+m*l*l))

j21 = sympy.diff(e2_d, e1)
j22 = sympy.diff(e2_d, e2)
j23 = sympy.diff(e2_d, e3)
j24 = sympy.diff(e2_d, e4)

j31 = sympy.diff(x3_d, e1)
j32 = sympy.diff(x3_d, e2)
j33 = sympy.diff(x3_d, e3)
j34 = sympy.diff(x3_d, e4)

JM = sympy.Matrix([[0,1,0,0],
                   [j21,j22,j23,j24],
                   [j31,j32,j33,j34],
                   [0,0,-kp2,-kd2]])

In [7]:
JM.evalf(subs={'e1':0, 'e2':0, 'e3':0, 'e4':0})

Matrix([
[               0,               1.0,                0,                0],
[            24.5, -4.16666666666667,            150.0,           197.95],
[-7.0148115942029, -1.33700483091788, 70.3973913043478, 92.4663081159421],
[               0,                 0,            -60.0,           -79.18]])

In [8]:
JM.evalf(subs={'e1':0, 'e2':0, 'e3':0, 'e4':0}).eigenvects()

[(-4.05159651158116 - 11.2427101022432*I,
  1,
  [Matrix([
   [-0.0490905319207521 + 0.0470369605093297*I],
   [   0.727717938978839 + 0.361335834034951*I],
   [   0.341702265140175 + 0.299480216790605*I],
   [  -0.231909518015983 - 0.273879165974994*I]])]),
 (-4.05159651158116 + 11.2427101022432*I,
  1,
  [Matrix([
   [0.035510186691786 + 0.0626348784685531*I],
   [-0.848058729437214 + 0.145459479545785*I],
   [-0.437652186803778 + 0.199996757065235*I],
   [ 0.318488928703933 - 0.207384735954201*I]])]),
 (-3.60450755972204 + 3.32070499780566e-62*I,
  1,
  [Matrix([
   [-0.100334338172525 - 0.135094777833464*I],
   [ 0.361655880442574 + 0.486950147979691*I],
   [-0.372023319635298 - 0.500909345996572*I],
   [ 0.295352348458158 + 0.397676016250167*I]])]),
 (-1.24157477943445 - 2.34754303418546e-62*I,
  1,
  [Matrix([
   [ 0.0801252007024859 + 0.0290490728069774*I],
   [-0.0994814283893299 - 0.0360665961630983*I],
   [   0.943306673170374 + 0.341992082241158*I],
   [   -0.72619378990593 

# controllability matrix

In [3]:
Y = sympy.Matrix([[0, 1, 0, 0],
              [A, B, 0, C],
              [0, 0, 0, 1],
              [E, F, 0, G]])
Z = sympy.Matrix([[0],
              [D],
              [0],
              [H]])
controllability = sympy.Matrix([Z,Y*Z,Y*Y*Z,Y*Y*Y*Z]).reshape(4,4)

In [4]:
controllability.rank() # full-rank means controllable

4